#### Step 1

0. defining location
1. defining peak power
2. choosing the module
3. choosing the inverter
4. choosing the number of modules in series (n_series)
5. choosing the number of series in parallel (n_strings)
6. suggest an ideal number of inverters
7. choose number of inverters

1) Goals & site

Location: Latitude/Longitude (map pin), auto-fill altitude (pvgis).
Target peak power (kWp) and Max area (m²) (optional).
Tech type (dropdown): TechChoice.
Tracking type (dropdown): TrackingType.
Mounting place (dropdown): RackingType
Azimuth (dial/compass) & Tilt (slider).
Surface type (dropdown → sets albedo): SurfaceType

In [1]:
from pvlib.pvsystem import retrieve_sam

from models.pv_custom.enums import RackingType, SurfaceType, TechChoice, TrackingType
from models.pv_custom.pv_plant import PVPlant

# input after selecting kwp, lat, lon, tech_choice
pv_plant = PVPlant(
    uri='',
    pod='IT000E00092',
    pod_uri='paidsfhgapghadg',
    kwp=6,
    maximum_area=0,
    latitude=41.9028,
    longitude=12.4964,
    angle=21.2,
    aspect=0,
    tech_choice=TechChoice.Mono_c_Si,
    tracking_type=TrackingType.fixed,
    mounting_place=RackingType.semi_integrated,
    surface_type=SurfaceType.concrete,
    percentage_loss=17,
    yearly_loss=0.5,
)

In [2]:
from apis.pvlib_helper.geographical_helper import GeographicalHelper
from apis.pvlib_helper.pvsystems_helper import PVSystemHelper


cecmodules = retrieve_sam('cecmod')
inverters = retrieve_sam('cecinverter')
altitude = GeographicalHelper.get_elevation_opentopo(lat=pv_plant.latitude, lon=pv_plant.longitude)
hot_temperature, cold_temperature = GeographicalHelper.get_design_temperatures(
    lat=pv_plant.latitude, 
    lon=pv_plant.longitude, 
    altitude=altitude,
    surface_type=pv_plant.surface_type.name if pv_plant.surface_type is not None else None,
    racking_type=pv_plant.mounting_place.name,
    tilt_angle=pv_plant.angle
    )
filtered_modules = PVSystemHelper.filter_cec_modules(pv_plant.get_tech_choice_description)
filtered_inverters = PVSystemHelper.filter_cec_inverters_by_modules(filtered_modules, hot_temperature=hot_temperature, cold_temperature=cold_temperature)

pv_plant.set_altitude(altitude)
pv_plant.set_module_names(filtered_modules.columns)
pv_plant.set_inverter_names(filtered_inverters.columns)



In [5]:
hot_temperature

np.float64(115.2)

#### Step 2

0. choose location
1. define surface type
2. fetch and preprocess (extract ghi, dhi, dni, airmass, precipitable_water) meteo data

#### Step 3

0. define TRACKING_TYPE (fixed, single axis, two axes)
1. define RACKING_MODEL
2. define TILT (used both for fixed modules and single axis tracking)
3. define AZIMUTH (used both for fixed modules and single axis tracking)
4. define MAX_ANGLE if it is single axis tracking
5. define BACKTRACK if it is single axis tracking (True vs False)
6. define GCR if it is single axis tracking (default 0.35)
7. define temperature parameters (rules for default parameters is defined in class RackingType)
8. define b0 parameter of the module for IAM coefficient (defulat 0.05) for the ashrae aoi model

#### Step 4

0. define wiring material (WiringMaterial enum)
1. define wiring length
2. define wiring cross section
3. define AC_WIRING_FACTOR
4. define AVAILABILITY
5. define LID_FACTOR
6. define MODULE_QUALITY_FACTOR
7. define MISMATCH_FACTOR
8. extract SOILING_MONTHLY (Monthly soiling transmittance)

Step 5

0. define pvlib.pvsystem.Array
1. define pvlib.pvsystem.PVSystem
2. define pvlib.modelchain.ModelChain
3. apply posterior loss model
4. multiply by the number of inverters

#### Step 6

0. Apply Markov Chain Montecarlo model to the 12 years of historical backcast of production
1. Save the ProductionMatrix
2. Simulate self consumption and grid injection